# Custom Chatbot Project

To demonstrate that the estimation performance of a large language model can be greatly improved without any further training, this notebook takes an older model from OpenAI and data that is not likely available in its training. Eventually the inference of the vanilla modus and the enhanced version are compared. In order to avoid that the data is present in the model's training data, a relatively new and non-popular topic is chosen.

The data will be about Zynthian user guides. Zynthian (https://zynthian.org/) is a young open source project that turns a Raspberry Pi into a synthesizer instrument.

The Zynthian user guide data is shared as creative commons license CC BY-SA 3.0 and so is this code 

[![Creative Commons](https://wiki.zynthian.org/resources/assets/licenses/cc-by-sa.png)](https://creativecommons.org/licenses/by-sa/3.0/)




## Data Wrangling

First the Zynthian focused information need to get downloaded, cleaned and prepared so that they eventually can get ranked and sorted by relevance of arbitrary questions.

In [1]:
import os
import json
import requests
import tiktoken
import pandas as pd
from bs4 import BeautifulSoup
from openai.embeddings_utils import get_embedding, distances_from_embeddings
import openai

Define notebook wide common settings for interacting with OpenAI's model.

__Important Note__: please add your API key to the "open_ai_key.txt" file or assign it directly to the `openai.api_key` parameter below

In [2]:
tokenizer = tiktoken.get_encoding("cl100k_base")
embedding_model_name = "text-embedding-ada-002"
completion_model_name = "gpt-3.5-turbo-instruct"

openai.api_base = "https://openai.vocareum.com/v1"

with open("open_ai_key.txt", "r") as keyfile:
    key_str = keyfile.read().replace("\n", "")
    assert len(key_str) > 0, "Can't continue without specifying a API key for OpenAI"
openai.api_key = key_str

### Get the raw context data

Zynthian offers a Wiki where the user guides can be fetched from. This HTML data needs to be converted into human readable strings, and cleaned from dublicates or even empty entries.

In [3]:
def extract_zynthian_context() -> pd.DataFrame:
    """
    This function provides essential Zynthian user guide information, 
    extracted from all <p></p> sections of its Wiki.
    
    Args:
        None
    Returns:
        pd.DataFrame: cleaned DataFrame with the extracted data in the "text" column name
    """
    
    # define set of webpages with user guides
    landing_page = "https://wiki.zynthian.org/index.php"
    user_guide_pages =[
        "Zynthian_UI_User%27s_Guide_-_Oram",
        "ZynSeq_User_Guide",
        "ZynSampler_User_Guide",
        "Web_Configuration_User_Guide",
        "Supported_plug_%26_play_MIDI_controllers",
    ]

    # download & extract information from said web pages
    all_paragraphs = list()
    for user_guide_page in user_guide_pages:
        subpage_response = requests.get(os.path.join(landing_page, user_guide_page))
        soup = BeautifulSoup(subpage_response.text)
        user_guide_p = soup.find_all("p")    

        for cur_p in user_guide_p:
            all_paragraphs.append(cur_p.text.replace("\n", ""))


    # turn it to a dataframe and apply data cleaning
    df = pd.DataFrame()
    df["text"] = all_paragraphs
    df = df[df["text"] != ""]
    df = df.drop_duplicates()
    
    return df
    
context_df = extract_zynthian_context()
context_df  # let's peek into the raw data

,text
0,V5 |V4 |Touch
1,Zynthian Oram is the most recent version of zy...
2,It is strongly recommended that you read secti...
3,"This guide is a living document, subject to fr..."
4,The fundamental building block of zynthian's s...
...,...
482,NOTE: Remember to put PAD CONTROLS in CC mode....
483,"In this mode, all notes of the KeyBed are used..."
484,"For other settings, you can use the Knobs. Mov..."
488,It works much like the Launchpad Mini MK3.


## Query Completion

In this section the mechanisms are prepared in order to provide a prompt with a meaningful context, where the LLM can extract valuable information for the given user question.

### Calculating Embeddings

Embeddings are necessary for model inference but we're also using it to compare the similarity of a given user-question with the Zynthian user-guides.

In [4]:
def attach_embeddings(df: pd.DataFrame, embedding_model_name:str, batch_size=100) -> pd.DataFrame:
    """
    Takes a DataFrame with raw context information and calculates its embeddings and returns the updated frame.
    
    Args:
        df: DataFrame with human readable strings in column "text" that should be encoded
        embedding_model_name: in what manner OpenAI should encode the data
    Returns:
        pd.DataFrame: DataFrame with "text" and "embedding" columns
    """
    
    embeddings = []
    batch_size = 100  # iterate batch-wise to avoid API-overstraining
    for i in range(0, len(df), batch_size):
        # Actual embeddings will be calculated by OpenAI and applied for via its API
        response = openai.Embedding.create(
            input=df.iloc[i:i+batch_size]["text"].tolist(),
            engine=embedding_model_name
        )

        # Turn OpenAI packet into a list of embeddings
        embeddings.extend([data["embedding"] for data in response["data"]])

    df["embeddings"] = embeddings
    return df

context_with_embeddings_df = attach_embeddings(context_df, embedding_model_name)
context_with_embeddings_df


,text,embeddings
0,V5 |V4 |Touch,"[-0.009096662513911724, -0.007616293150931597,..."
1,Zynthian Oram is the most recent version of zy...,"[-0.029481329023838043, -0.016066910699009895,..."
2,It is strongly recommended that you read secti...,"[-0.010694820433855057, 0.010080959647893906, ..."
3,"This guide is a living document, subject to fr...","[-0.0006172802532091737, -0.000813496182672679..."
4,The fundamental building block of zynthian's s...,"[-0.008502153679728508, -0.005421128589659929,..."
...,...,...
482,NOTE: Remember to put PAD CONTROLS in CC mode....,"[-0.020462140440940857, -0.013787965290248394,..."
483,"In this mode, all notes of the KeyBed are used...","[-0.01658054068684578, 0.002622124506160617, 0..."
484,"For other settings, you can use the Knobs. Mov...","[0.009522629901766777, -0.01551835983991623, -..."
488,It works much like the Launchpad Mini MK3.,"[-0.009409577585756779, 0.012401715852320194, ..."


### Prepare temporary context

The user guide information was downloaded and stored in an arbitrary fashion. OpenAI's model interface is limited by the total number of tokens in a prompt. It's because of this we can't use the full extracted information and have to prepare a subset of most relevant information. This can be achieved by encoding embeddings of the context paragarphs and measure its likeliness to the encoded question like in the following.

In [5]:
def get_relevant_context(question:str, df:pd.DataFrame, embedding_model_name:str, metric:str="cosine") -> pd.DataFrame:
    """
    This function takes the unordered context data and orders it by relevance 
    according to the given question.
    
    Args:
        question: the string that is used to order the dataset by likeliness
        df: the unordered DataFrame with "text" and "embeddings" columns
        embedding_model_name: in what manner OpenAI should encode the data
    Returns:
        pd.DataFrame: a DataFrame sorted by relevance to the given question
    
    """
    
    # Call OpenAI's API to calculate the relevance parameter and attach it to the dataframe
    df["distances"] = distances_from_embeddings(
        query_embedding=get_embedding(question, engine=embedding_model_name),
        embeddings=df["embeddings"].values,
        distance_metric=metric
    )
    
    # Take the unordered DataFrame and sort it by the just calculcated relevance parameter
    return df.sort_values("distances", ascending=True)

test_question = "What is known about tempo?"
relevant_context = get_relevant_context(test_question, context_with_embeddings_df, embedding_model_name)

n_check_entries = 3
print(f"Q: {test_question}\n")
print(f"[info] Unorderd DataFrame\n{context_with_embeddings_df[0:n_check_entries]['text']}\n")
print(f"[info] DataFrame ordered by relevance\n{relevant_context[0:n_check_entries]['text']}\n")

Q: What is known about tempo?

[info] Unorderd DataFrame
0                                        V5 |V4 |Touch
1    Zynthian Oram is the most recent version of zy...
2    It is strongly recommended that you read secti...
Name: text, dtype: object

[info] DataFrame ordered by relevance
265    The current tempo is saved and loaded with eac...
264    Tempo is the rate at which the sequencer plays...
240    Tempo may be adjusted using the SNAPSHOT encod...
Name: text, dtype: object



## Prompt generator

In order that we can use the chat bot with the extended context information, a standardized way of creating its prompt is being prepared here. It carefully watches the number of utilized token doesn't exceed its limits, puts the relevant context and eventually the actual query into one large string

In [6]:
def make_prompt(question, 
                context_df,
                embedding_model_name,
                max_question_len=100, 
                max_prompt_tokens=1800,
                verbose=False):
    """
    This function provides standardized prompts. The context is ordered by relevance to the user query.
    
    Args:
        question: the string that is used to order the dataset by likeliness
        context_df: the unordered DataFrame with "text" and "embeddings" columns
        embedding_model_name: in what manner OpenAI should encode the data
        max_question_len: the number of characters shouldn't exceed this number
        max_prompt_tokens: not more than this number of tokens will be passed to OpenAI and exiting
            early the context creation if this value is exceeded
    Returns:
        str: the prompt with relevant context and user question
    
    """
    assert len(question) < max_question_len, f"Your question is too long, please rephrase to use less than {max_question_len} charachters."
    if verbose:
        print(f"[info] Making a prompt for: {question}")
    # set up raw layout of prompt
    prompt_context = "Context:\n"
    prompt_question = f"Question:\n{question}"
    relevant_context = get_relevant_context(question, context_df, embedding_model_name)
    
    # begin assembling the relevant context as long as there are unused tokens left
    remaining_tokens = len(tokenizer.encode(prompt_context)) + len(tokenizer.encode(prompt_question))
    for idx, text_element in enumerate(relevant_context["text"]):
        remaining_tokens += len(tokenizer.encode(text_element))
        if remaining_tokens > max_prompt_tokens:
            if verbose:
                print(f"[info] Limited context to {idx}/{relevant_context.shape[0]} paragraphs")
            break
        else:
            prompt_context += f"{text_element}\n---\n"
    
    # complete full prompt
    prompt = prompt_context + prompt_question
    
    if verbose:
        print("[info] Created following prompt:")
        print(f"===========================\n\n{prompt}\n\n")

    return prompt

_ = make_prompt(test_question, context_with_embeddings_df, embedding_model_name, verbose=True)

[info] Making a prompt for: What is known about tempo?
[info] Limited context to 30/386 paragraphs
[info] Created following prompt:

Context:
The current tempo is saved and loaded with each snapshot.
---
Tempo is the rate at which the sequencer plays back notes measured in beats per minutes (BPM). By default ZynSeq plays sequences at 120 BPM. Adjust Tempo with the SNAPSHOT encoder. The tempo is briefly displayed in the title bar. There is also a menu option to adjust tempo.
---
Tempo may be adjusted using the SNAPSHOT encoder or by selecting "Tempo" from the menu.
---
ZynSeq allows tempo to be adjusted from 1.0 BPM to 420.0 BPM in 0.1 BPM steps. Tempo may also be altered by external modules, e.g. MIDI player.
---
Knob K3 is used to adjust the tempo. When rotated, the Zynthian tempo screen will be shown briefly. This tempo setting is synchronized with the MPK. 
---
For instance, if you are in the mixer view and short-push OPT/ADMIN, the Main menu will be opened. If you short-push it aga

### Prompt submission
After all preparations have been made to provide a prompt of higher quality, a standardized way of querying OpenAI's model with that prompt is established first before eventually running a test comparison.

In [7]:
def ask_openai(prompt:str, completion_model_name:str, max_answer_tokens:int=200) -> str:
    """
    This submits arbitrary prompts to OpenAI's Completion interface and 
    returns the most likely estimate as a string
    
    Args:
        prompt: the query that will be passed to the model
        completion_model_name: the model that the query should interfere with
        max_answer_tokens: limitation to avoid overstraining OpenAI services
    Returns:
        str: the most likely answer for the input question    
    """  
    try:
        response = openai.Completion.create(
            model=completion_model_name,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        # return most likely answer
        return response["choices"][0]["text"].strip()
    # Embedding this in a sand box if for instance the API is not reachable.
    except Exception as e: 
        print(e)
        return ""

## Custom Performance Demonstration

Here the performance is demonstrated by two questions:

- the vanilla modus just takes the question and queries OpenAI's Completion model.
- the context modus takes the question and adds, relative to the given question, relevant information from the Zynthian user guides.

Two questions are being picked that are less likely to be answered by general knowledge of synthesizers

### Question 1

This question covers a couple of paragraphs referencing the unit, that makes guessing harder

In [8]:
question = "List all functions of the 'short-push' button in Zynthian."

In [9]:
vanilla_prompt = question
print(ask_openai(vanilla_prompt, completion_model_name))

1. Selecting a menu option: The short-push button can be used to select a menu option on the Zynthian display. This is useful for navigating through different menus and sub-menus.

2. Confirming a selection: Once a menu option has been selected, the short-push button can be used to confirm the selection and execute the command associated with it.

3. Scrolling through lists: In certain menus or sub-menus, the short-push button can be used to scroll through a list of options. This is typically used when there are more options than can fit on the display at once.

4. Muting/unmuting a layer: When playing a synthesizer or sampler layer, the short-push button can be used to mute or unmute that layer. This is useful for creating variations in a performance or for soloing a specific layer.

5. Setting the tempo: Pressing and holding the short-push button for a few seconds will allow you to adjust the tempo of


In [10]:
context_prompt = make_prompt(question, context_with_embeddings_df, embedding_model_name)
print(ask_openai(context_prompt, completion_model_name))

1. Access the zynthian's classic workflow (V1-V4).
2. Contextual global actions.
3. Show onscreen buttons.
4. View basic configuration information on the dashboard.
5. Control the Zynthian UI using Shift+Device.
6. Blink the device button in red when the mode is active.
7. Redefine the functionality of transport, arrows, and F1-F4 buttons.
8. Mimic the Zynthian V5 hardware interface.
9. Enable transport buttons for playback/recording status.
10. Use the UP, DOWN, LEFT, RIGHT, BACK/NO, and SEL/YES buttons for navigation.
11. Use track keys as directional keys.
12. Use a MIDI keyboard to trigger a sequence.
13. Configure Zynthian interface options in webconf.
14. Update the Zynthian software.
15. Preview a pattern and adjust tempo with the encoder or menu.
16. Use the nearest four knobs as Knob


### Question 2

For verification see https://wiki.zynthian.org/index.php/ZynSeq_User_Guide#Time_Signature

In [11]:
question = "In Zynthian: what is the most significant behaviour that influences beats per bar ?"
vanilla_prompt = question
print(ask_openai(vanilla_prompt, completion_model_name))

The most significant behavior that influences beats per bar in Zynthian is the tempo setting. This determines the speed at which the beats per bar are played. Other factors such as the time signature and the type of rhythm or pattern being used can also affect the beats per bar, but the tempo is the main factor that determines the overall speed of the beats.


In [12]:
context_prompt = make_prompt(question, context_with_embeddings_df, embedding_model_name)
print(ask_openai(context_prompt, completion_model_name))

Answer:
The sync point.
